A chatbot is an intelligent piece of software that is capable of communicating and performing actions similar to a human.

It is of two types:
 - Retrieval based models
 - Generative based models.

### Retrieval-Based Models

**What They Do:** Retrieval-based chatbots look through a predefined set of responses and pick the best one based on the user's input. 

**How They Work:**
1. **Predefined Responses:** They have a list of possible answers stored in advance.
2. **Matching:** When a user asks something, the chatbot matches the question to the most relevant response from its list.
3. **Selection:** It then selects and displays the best response it found.

**Example:** If you ask a retrieval-based chatbot, "What’s the weather today?" it might look up responses related to weather queries and choose a pre-written response about today’s weather.

**Pros:** 
- Simple to implement.
- Good at providing accurate responses if questions are predictable.

**Cons:** 
- Limited by the responses it has.
- Cannot handle unexpected questions well.



### Generative-Based Models

**What They Do:** Generative-based chatbots create responses from scratch using learned patterns from data. They generate replies based on the input they receive.

**How They Work:**
1. **Learning from Data:** They are trained on large amounts of conversation data.
2. **Generating Responses:** When you ask a question, the chatbot generates a new, unique response based on what it has learned.

**Example:** If you ask a generative-based chatbot, "Tell me a joke," it might create a new joke based on patterns it has learned from previous jokes.

**Pros:**
- Can handle a wide range of topics and questions.
- Produces more natural and varied responses.

**Cons:**
- More complex to build.
- Responses might be less predictable and require careful tuning.

In [4]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

FileNotFoundError: [Errno 2] No such file or directory: 'intents.json'

In [3]:
print("Hello world")

Hello world
